# An example of failure of GMRES method and convergence of the SMW iteration

### 18 July 2023
### Dimitrios Mitsotakis

In [1]:
import numpy as np
import scipy as sp
import scipy.sparse as sps
import scipy.sparse.linalg as spsl
import numpy.linalg as npl
import scipy.linalg as spl
from numpy.fft import fft, ifft

A trivial implementation of the SMW iteration:

In [2]:
def SMW_iteration(M, N, b, x, tol = 1.e-5, maxit = 100):
    # M, N : matrices such as A = M - N
    # M : is the first column of the circulant matrix M
    # N : is the sparse matrix N
    # x : guess of the solution
    
    err = 1.0
    iters = 0
    
    while (err > tol and iters < maxit):
        iters += 1
        c = N@x+b
        xnew = spl.solve_circulant(M,c)
        err = npl.norm(xnew-x)
        x = np.copy(xnew)
        
    print('iterations required for convergence:', iters)
    
    return x

This is the setup of a dense system. Try the last two commands only if you take `n` reasonably small.

In [3]:
from numpy.random import rand

n = 1000000

M = 1.0+rand(n)
xexact = np.ones((n,1))
N = sps.lil_matrix((n, n))
N[0,0]=-1.0; N[-1,0]=-1.0; N[0,-1]=-1.0; N[-1,-1]=-1.0;

s = np.sum(M)
b = np.ones((n,1))*s
b[0] -= N[0,0] + N[0,-1]
b[-1] -= N[-1,0] + N[-1,-1]

xexact = np.ones((n,1))
x0 = np.zeros((n,1))

# A = spl.circulant(M)-N
# print('condition number=',npl.cond(A))


In [4]:
# This is our method:

x = SMW_iteration(M, N, b, x0, tol = 1.e-8, maxit = 100)

print('The error is:', npl.norm(x-xexact)) 

iterations required for convergence: 4
The error is: 1.0738412017708304e-09


In the following code we test the python GMRES implementation. I wouldn't try `n=1000000` but you are welcome to do if you like!

In [5]:
# This is the GMRES:

from scipy.sparse.linalg import gmres

n = 100

M = 1.0+rand(n)
xexact = np.ones((n,1))
N = sps.lil_matrix((n, n))
N[0,0]=-1.0; N[-1,0]=-1.0; N[0,-1]=-1.0; N[-1,-1]=-1.0;

s = np.sum(M)
b = np.ones((n,1))*s
b[0] -= N[0,0] + N[0,-1]
b[-1] -= N[-1,0] + N[-1,-1]

xexact = np.ones((n,1))
x0 = np.zeros((n,1))

A = spl.circulant(M)-N
print('condition number=',npl.cond(A))


%time (x, info) = gmres(A, b, x0, tol=1.e-8, maxiter=1000, atol=1.e-8)

print(info)
print('The error is:', npl.norm(x-xexact)) 

condition number= 5638.1811061351755
CPU times: user 1.69 s, sys: 308 ms, total: 2 s
Wall time: 513 ms
1000
The error is: 0.1316396494895858
